In [2]:
from math import *
import numpy as np
from matplotlib import pyplot as plt
from collections import namedtuple as nt

# Forwards compatible storage

The issue is that 

1. we have a fixed and limited amount of storage to store our curve data
2. our current precision is not sufficient and needs to be increased,
3. we do not want to rush what is an important and complex issue, and
4. we want storage to be forwards compatible*

*forwards compatible in this context means that any storage records currently created, when fed into the new system, must be understood as is, without any additional information like a versioning record

Caveat: this is based on my understanding of languages like C; if Solidity behave massively different then this may not be apply; also Python is not the perfect way of demonstrating this as Python has unlimited-size integers, but the principle should be clear

## Variable decomposition

Here we first consider how we can combine at 112 bit variable and a 16 bit variable into a 128 bit variable. We also decompose an 80 bit variable into a 16 bit variable and a 64 bit variable. Note that those are the same operations, they are just named differently because the purpose is different. 

In [23]:
nt_112_16 = nt("r", "i112, i16")
nt_16_64 = nt("r", "i16, i64")
def combine_112_16(i112, i16):
    """
    combines a 112 bit uint and a 16 bit uint into a 128 bit unsigned int
    """
    assert i112 >= 0
    assert i16 >= 0
    assert i112 < (1<<112)
    assert i16 < (1<<16)
    result = i112 + (1<<112) * i16
    assert result < (1<<128)
    return result
    #return result, log2(result) if result > 0 else None

def extract_112_16(i128):
    """extracts a 112 bit uint and a 16 bit uint from a 128 bit uint"""
    assert i128 >= 0
    assert i128 < (1<<129)
    i112 = i128 & ((1<<112)-1)
    i16 = (i128>>112) & ((1<<16)-1)
    return nt_112_16(i112,i16)

def decompose_i80(i80):
    """decomposes uint i80 from uint i16, i64 (no checks)"""
    i16 = i80 & ((1<<16)-1)
    i64 = (i80>>16) & ((1<<64)-1)
    return nt_16_64(i16, i64)
    
def reconstitute_i80(i16, i64):
    """reconstitutes 80 bit uint from 16 and 64 bit uint (no checks)"""
    return i16 + (i64<<16)

In [4]:
c = combine_112_16(1,60000)
extract_112_16(c), c

(r(i112=1, i16=60000), 311537811512089657711829779753205760001)

In [5]:
c = combine_112_16(1,0)
extract_112_16(c), c

(r(i112=1, i16=0), 1)

In [6]:
c = combine_112_16(1,1)
extract_112_16(c), c

(r(i112=1, i16=1), 5192296858534827628530496329220097)

In [24]:
c = decompose_i80(123456789)
c, reconstitute_i80(*c)

(r(i16=52501, i64=1883), 123456789)

## Application to curve storage

For simplicity we only consider one half of the problem, so let's say we have one variable `z` (112 bit) and one variable `A` (80 bit) and we have two memory slots of 128 bit and 64 respectively. 

**The purpose here is to show how we can abstract the storage layout (128+64bit) from the variable layout (112+80bit)**

In [8]:
nt_128_64 = nt("r", "i128, i64")
nt_z_A = nt("r", "z,A")
def to_storage(z, A):
    """saves z (112 bit) and A (80 bit) into a storage of i128, i64 of 128 and 64 bits respectively"""
    assert z >= 0
    assert A >= 0
    assert z <  1<<112
    assert A <  1<<80
    A_16, A_64 = decompose_i80(A)
    return nt_128_64(combine_112_16(z, A_16), A_64)

def from_storage(i128, i64):
    """reverts the to_storage function"""
    z, A_16 = extract_112_16(i128)
    A = reconstitute_i80(A_16, i64)
    return nt_z_A(z,A)

In [9]:
c = to_storage(123456,789)
from_storage(*c), c

(r(z=123456, A=51707904), r(i128=4096722221383978998910561603754779200, i64=0))

In [10]:
c = to_storage(12,345678)
from_storage(*c), c

(r(z=12, A=1179844608), r(i128=93450958859909827658291872933303287820, i64=5))

In [11]:
c0,c1 = 2**111+123, 12345
c = to_storage(c0,c1)
from_storage(*c), c, c0, c1

(r(z=2596148429267413814265248164610171, A=809041920),
 r(i128=64101500867041714488023242432386695291, i64=0),
 2596148429267413814265248164610171,
 12345)

## Future compatible curve storage

Here our logical storage is separated into 3 variables

- `z` as before
- `A` the value of A under the current implementation
- `A_incr` the incremental information for A under the future implementation

**The future implementation will interpret `(A, 0)` in excactly the same way the current implementation interprets `A`.**

In [12]:
nt_z_A_A = nt("r", "z,A,A_incr")
def to_storage_fc(z, A, A_incr=0):
    """packs z (112 bit), A (64 bit) and A_incr (16bit, optional) into storage"""
    return nt_128_64(combine_112_16(z, A_incr), A)

def from_storage_fc(i128, i64):
    """reverts the to_storage_fc function"""
    z, A_incr = extract_112_16(i128)
    A = i64
    return nt_z_A_A(z, A, A_incr)

In [13]:
c=to_storage_fc(123,456)
from_storage_fc(*c), c

(r(z=123, A=456, A_incr=0), r(i128=123, i64=456))

In [14]:
c=to_storage_fc(123, 456, 78)
from_storage_fc(*c), c

(r(z=123, A=456, A_incr=78),
 r(i128=404999154965716555025378713679167611, i64=456))

## Floating point storage

When storing A, B the issue is not that much the number of bits we need for any given curve: we have 64 bits which corresponds to 10^10 which is more than enough to cover all possible prices of a given pair. The problem arises when we want to cover _all possible pair_, especially in a crypto world where prices need to be scaled with 10^ddec where ddec is the difference in decimals between the two tokens. Decimal values can go from 0 to 24, with the core range being from 6 to 18, so this factor can be from 10^12 to 10^24 in the worst case.

In order to store only the scaling factors we need at least 40 bit (18/6) and at worst 80 bit (24/0) case. On top of this we need at least 16 bit for the actual price (10^5), but better 25 bit (10^7). On the face of it this means that we need at least 56-65 bit [which is why we are borderline right now, with 64 bit], but in order to be safe for all edge cases we need 96-115 bit.

However, in reality this is not the case. The reason for this is **that it is not optimal to store the scaling factor as a standard int; instead the exponent should be stored.** As a reminder our scaling factor had to cover 2^0 to 2^80 in the worst case scenario, so 6 bit is a bit tight, but 7 or 8 bit is ample to cover all possible scaling factors and more.


In [15]:
2**64, log10(2**64)

(18446744073709551616, 19.265919722494797)

In [16]:
log2(10**12), log2(10**24), log2(10**5), log2(10**7), log2(80)

(39.86313713864835,
 79.7262742772967,
 16.609640474436812,
 23.253496664211536,
 6.321928094887363)

In [25]:
def scaling_factor_from_store(a, aexp):
    """returns scaling factor based on a and exponent aexp"""
    return a << aexp

In [26]:
scaling_factor_from_store(123456, 12)

505675776